## Importing

In [1]:
import argparse
import math
import time

import torch
import yaml
import numpy as np

from audit import audit_models, audit_records, sample_auditing_dataset, exp_estimated_epsilon
from get_signals import get_model_signals
from models.utils import load_models, train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories,
    load_dataset,
    load_subset_dataset,
)

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

/data/yx23/anaconda3/envs/privacy_meter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load config

In [2]:
configs = "configs/tabular/adult/adult_dpsgdfga.yaml"
with open(configs, "rb") as f:
    configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
subdata_dir = configs["train"]["log_dir"] 
log_dir = subdata_dir + configs["train"]["model_name"] + "/" + configs["train"]["method"]
if configs["train"]["epsilon"] > 0:
    log_dir += f"/eps{int(configs["train"]["epsilon"])}"
configs["train"]["log_dir"] = log_dir
configs["run"]["log_dir"] = log_dir

directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    #"signal_dir": f"{log_dir}/signals",
    "data_dir": configs["data"]["data_dir"],
    "subdata_dir": subdata_dir,
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

In [3]:
log_dir

'runs/demo_tabular/adult/LR/dpsgdfga/eps10'

## Load dataset

In [4]:
baseline_time = time.time()
dataset = load_dataset(configs, directories["data_dir"], logger)
logger.info("Loading dataset took %0.5f seconds", time.time() - baseline_time)

# # subset of dataset
# if configs["train"]["data_size"] < len(dataset):
#     dataset = load_subset_dataset(configs, dataset, f"{directories["subdata_dir"]}", logger)
#     logger.info("Loading sub-dataset took %0.5f seconds", time.time() - baseline_time)

2024-12-23 20:29:33,100 INFO     Load data from data/tabular/adult.pkl
12/23/2024 20:29:33:INFO:Load data from data/tabular/adult.pkl
2024-12-23 20:29:33,102 INFO     The whole dataset size: 45222
12/23/2024 20:29:33:INFO:The whole dataset size: 45222
2024-12-23 20:29:33,103 INFO     Loading dataset took 0.02563 seconds
12/23/2024 20:29:33:INFO:Loading dataset took 0.02563 seconds


## Load or train models

In [5]:
# Define experiment parameters
model_num = 1
num_model_pairs = 1

# Load or train models
baseline_time = time.time()
models_list, memberships = load_models(
    log_dir, dataset, model_num, configs, logger
)


2024-12-17 21:03:10,327 INFO     Loading model 0
12/17/2024 21:03:10:INFO:Loading model 0


In [ ]:
if models_list is None:
    data_splits, memberships = split_dataset_for_training(
        len(dataset), num_model_pairs, ratio=0.5
    )
    data_splits.pop()
    memberships = memberships[:-1]
    models_list = train_models(
        log_dir, dataset, data_splits, memberships[:-1], configs, logger
    )
logger.info(
    "Model loading/training took %0.1f seconds", time.time() - baseline_time
)

## Prepare auditing dataset

In [13]:
memberships = np.array([memberships])

In [15]:
# auditing_dataset, auditing_membership = sample_auditing_dataset(
#         configs, dataset, logger, memberships
#     )
auditing_dataset, auditing_membership = dataset, memberships

## Compute signals

In [9]:
baseline_time = time.time()
configs["audit"]["algorithm"] == "LOSS"
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

Computing softmax: 100%|██████████| 3/3 [00:00<00:00, 19.04it/s]
2024-12-17 21:03:49,067 INFO     Signals saved to disk.
12/17/2024 21:03:49:INFO:Signals saved to disk.
2024-12-17 21:03:49,071 INFO     Preparing signals took 2.16958 seconds
12/17/2024 21:03:49:INFO:Preparing signals took 2.16958 seconds


## Attack scores

In [16]:
# Perform the privacy audit
baseline_time = time.time()
#target_model_indices = list(range(num_experiments))
target_model_indices = list(range(model_num)) # for all models

# shape of mia_score_list: (n, m) n=(num_reference_models+1)*2, m=len(auditing_dataset)
mia_score_list, membership_list = audit_records(
    f"{directories['report_dir']}",
    target_model_indices,
    signals,
    auditing_membership,
    model_num,
    logger,
    attack_algorithm='LOSS',
)

2024-12-17 21:04:41,045 INFO     mia_score_list and membership_list saved to disk.
12/17/2024 21:04:41:INFO:mia_score_list and membership_list saved to disk.


In [17]:
mia_score_list.flatten(), membership_list.flatten().astype(int)

(array([-0.13504328, -0.07227418, -0.06299163, ..., -0.05772954,
        -0.10837515, -0.1222467 ], dtype=float32),
 array([0, 0, 1, ..., 1, 1, 0]))

In [20]:
#算auc
from sklearn.metrics import roc_curve, auc
all_auc = {i:[] for i in range(10)}
targets = []
for i,j in dataset:
    targets.append(j)
targets = np.array(targets)

s = mia_score_list.flatten()
m = membership_list.flatten().astype(int)
fpr_list, tpr_list, _ = roc_curve(m, s)
for i in range(10):
    fpr_list, tpr_list, _ = roc_curve(m[targets==i], s[targets==i])
    roc_auc = auc(fpr_list, tpr_list)   
    all_auc[i].append(roc_auc)

#每一类的平均auc
for i in range(10):
    print(np.mean(all_auc[i]))

0.5033595360718892
0.5090255215289552
0.5043626448534424
0.49363147792706336
0.5010946262272444
0.4755399504307801
0.5020277667197452
0.5107132331270262
0.47030107490649325
0.5110708928454967
